In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from datetime import datetime

In [2]:
time_start = datetime.now()

inception_model_avg_pool_features = pd.read_csv('inception_model_avg_pool_features.csv', sep=',', header=None)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:00:25.843834


In [10]:
inception_model_avg_pool_features.shape

(40000, 2048)

In [17]:
?kmeans

In [18]:
time_start = datetime.now()

kmeans = KMeans(n_clusters=20, max_iter=3000, random_state=2019)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:00:00


In [6]:
time_start = datetime.now()

kmeans.fit(inception_model_avg_pool_features.values)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:03:24.086465


In [7]:
time_start = datetime.now()

output = kmeans.predict(inception_model_avg_pool_features.values)

print('This step took time:', datetime.now() - time_start)

This step took time: 0:00:06.246959


In [8]:
labels = ["cluster" + str(i) for i in range(20)]
cluster_dict = dict.fromkeys(labels, None) # dict with cluster number as keys and indices of relevant patches as values

for i in range(len(output)):
    if cluster_dict["cluster" + str(output[i])] == None:
        cluster_dict["cluster" + str(output[i])] = [i]
    else:
        cluster_dict["cluster" + str(output[i])].append(i)
        
[len(cluster_dict[key]) for key in cluster_dict]

[35525,
 215,
 289,
 308,
 208,
 227,
 152,
 267,
 165,
 269,
 269,
 170,
 171,
 235,
 257,
 301,
 238,
 236,
 249,
 249]